In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [9]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [10]:
train.head()

In [11]:
test.head()

In [12]:
test_shape = test.shape
train_shape = train.shape
print(test_shape)
print(train_shape)

In [13]:
test.describe()

In [14]:
train.describe()

In [15]:
def kesson_table(df):
    null_val = df.isnull().sum()
    percent = 100*df.isnull().sum()/len(df)
    kesson_table = pd.concat([null_val,percent], axis=1)
    kesson_table_ren_columns = kesson_table.rename(columns = {0: '欠損数', 1: '%'}) 
    return kesson_table_ren_columns
kesson_table(train)

In [16]:
kesson_table(test)

In [17]:
train["Age"] = train["Age"].fillna(train["Age"].median())

In [18]:
kesson_table(train)

In [19]:
train["Embarked"] = train["Embarked"].fillna("S")

In [20]:
kesson_table(train)

欠損データの処理が終わったので次はカテゴリデータの文字列を数字に変換する。<br>
今回の予想で使用する項目で文字列を値として持っているカラムは「Sex」と「Embarked」の2種類。<br>
これらを数字に変換。


In [21]:
train["Sex"][train["Sex"]== "male"] = 0
train["Sex"][train["Sex"]=="female"]=0
train["Embarked"][train["Embarked"] == "S"] = 0
train["Embarked"][train["Embarked"] == "C"] = 1
train["Embarked"][train["Embarked"] == "Q"] = 2

In [22]:
train.head(10)

SexのカラムとEmbarkedのカラムがともに数値に置き換えられていることが確認できる。

In [23]:
test["Age"] = test["Age"].fillna(test["Age"].median())
test["Sex"][test["Sex"]== "male"] = 0
test["Sex"][test["Sex"]=="female"] = 1
test["Embarked"][test["Embarked"]=="S"] = 0
test["Embarked"][test["Embarked"]=="C"] = 1
test["Embarked"][test["Embarked"]=="Q"] = 2
test.Fare[152] = test.Fare.median() 

In [24]:
test.head(10)

In [25]:
kesson_table(test)

データの確認および事前処理も終わったため、予測モデルとして弱学習器「決定木」を採用。<br>
予測モデルその１として「Pclass」「Sex」「Age」「Fare」の4つの項目を利用して「生存可否」を予測する。

In [26]:
from sklearn import tree
#trainの目的変数および説明変数を首都
target = train["Survived"] .values
features_one = train[["Pclass","Sex","Age","Fare"]].values
#決定木の作成
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)
#「test」の説明変数の値を取得
test_features = test[["Pclass","Sex","Age","Fare"]].values
#「test」の説明変数を利用して「my_tree_one」のモデルで予測
my_prediction = my_tree_one.predict(test_features)

はじめに決定木で利用するTarget及びFeatureの値をtrainから取得し格納する。
さらにDecisionTreeClassifier()を利用しmy_tree_oneという決定木モデルを作成。

最後に事前に前処理したtestよりtrainで利用したFeatureと同様の項目の値をtest_featureへ格納しpredict()を使い予測した。

In [27]:
#予測データのサイズを確認
my_prediction.shape

In [28]:
print(my_prediction)

In [70]:
#PassengerIDを取得
PassengerId = np.array(test["PassengerId"]).astype(int)

#my_prediction(予測データ)とPassengerIDをデータフレームに落とし込む
my_solution = pd.DataFrame(my_prediction, PassengerId, columns=["Survived"])

#my_tree_one.csvとして書き出し
my_solution.to_csv("my_tree_one.csv", index_label=["PassengerId"])


In [73]:
docker info